In [18]:
import os
import sys
module_path = os.path.abspath(os.path.join('..\\PythonModules\\'))                                           
if module_path not in sys.path:
    sys.path.append(module_path)
##import Bedrock_Client
##import pyodbc
import pandas as pd
import PredictorOptimize

In [19]:
##df = pd.read_sql_query('select * fROM ref.SHMI_Model_Predict_Statistics',Bedrock_Conn)
import Pickle_Utility
df =Pickle_Utility.read('shimi_predictor')


df.to_csv('shmi_predictor.csv')

In [79]:
import matplotlib.pyplot as plt
PREDICTOR = 'Intercept'
DIAGNOSIS_GROUP = '1'

new_df = df[(df['PREDICTOR']==PREDICTOR)&(df['DIAGNOSIS_GROUP']==DIAGNOSIS_GROUP)][['Date','PARAMETER_ESTIMATE']].sort_values(by='Date')


In [80]:
new_df.set_index('Date', inplace=True)

In [81]:
new_df.astype('float')

,PARAMETER_ESTIMATE
Date,
2019-05-13,-4.7593
2019-06-13,-4.7471
2019-07-13,-4.6321
2019-08-13,-4.6613
2019-09-13,-4.4920
2019-10-13,-4.7900
2019-11-13,-4.6420
2019-12-13,-4.7312
2020-01-13,-4.8876


In [82]:
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.ar_model import ARResults
import numpy
 
# create a difference transform of the dataset
def difference(dataset):
    diff = list()
    for i in range(1, len(dataset)):
        value = dataset[i] - dataset[i - 1]
        diff.append(value)
    return numpy.array(diff)
df_train = new_df.drop(new_df.tail(1).index, inplace=False) 
# load dataset
series = df_train.astype('float')
X = difference(series.values)
# fit model
model = AR(X)
model_fit = model.fit(maxlag=5, disp=False)
# save model to file
model_fit.save('ar_model.pkl')
# save the differenced dataset
numpy.save('ar_data.npy', X)
# save the last ob
numpy.save('ar_obs.npy', [series.values[-1]])

In [83]:
print(X)

[[ 0.0122]
 [ 0.115 ]
 [-0.0292]
 [ 0.1693]
 [-0.298 ]
 [ 0.148 ]
 [-0.0892]
 [-0.1564]
 [ 0.2477]]


In [84]:
print(sum(X)/len(X))

[0.01326667]


In [85]:
[series.values[-1]]

[array([-4.6399])]

In [86]:
# load the AR model from file
from statsmodels.tsa.ar_model import ARResults
import numpy
loaded = ARResults.load('ar_model.pkl')
print(loaded.params)
data = numpy.load('ar_data.npy')
last_ob = numpy.load('ar_obs.npy')
print(last_ob)

[-0.0354333  -0.66667201  0.22065944  0.50854369 -0.34165519  0.12682167]
[[-4.6399]]


In [87]:
# load AR model from file and make a one-step prediction
from statsmodels.tsa.ar_model import ARResults
import numpy
# load model
model = ARResults.load('ar_model.pkl')
data = numpy.load('ar_data.npy')
last_ob = numpy.load('ar_obs.npy')
# make prediction
predictions = model.predict(start=len(data), end=len(data))
# transform prediction
yhat = predictions[0] + last_ob[0]
print('Prediction: %f' % yhat)

Prediction: -5.008699


In [88]:
new_df.tail(1)

,PARAMETER_ESTIMATE
Date,
2020-03-13,-4.439


In [89]:
# AR example
from statsmodels.tsa.ar_model import AR
from random import random
# contrived dataset
data = df_train.astype('float')
# fit model
model = AR(data)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(data), len(data))
print(yhat)

10   -4.841359
dtype: float64
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [90]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from random import random
# contrived dataset
data = df_train.astype('float')
# fit model
model = SimpleExpSmoothing(data)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(data), len(data))
print(yhat)

10   -4.64213
dtype: float64
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [91]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from random import random
# contrived dataset
data = df_train.astype('float')
# fit model
model = ExponentialSmoothing(data)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(data), len(data))
print(yhat)

10   -4.64213
dtype: float64
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [92]:
data

,PARAMETER_ESTIMATE
Date,
2019-05-13,-4.7593
2019-06-13,-4.7471
2019-07-13,-4.6321
2019-08-13,-4.6613
2019-09-13,-4.4920
2019-10-13,-4.7900
2019-11-13,-4.6420
2019-12-13,-4.7312
2020-01-13,-4.8876


In [138]:
data = df_train.astype('float').values
	# data split
n_test = 3
# model configs
cfg_list = PredictorOptimize.exp_smoothing_configs(seasonal=[None])
# grid search

#walk_forward_validation(data, n_test, cfg)

scores = PredictorOptimize.grid_search(data[:,0], cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

> Model[['add', True, None, None, False, True]] 0.623
 > Model[['add', True, None, None, False, False]] 0.175
 > Model[['add', False, None, None, False, True]] 0.623
 > Model[['add', False, None, None, False, False]] 0.186
 > Model[[None, False, None, None, False, True]] 0.549
 > Model[[None, False, None, None, False, False]] 0.176
done
['add', True, None, None, False, False] 0.1754870186730296
[None, False, None, None, False, False] 0.17629251659641273
['add', False, None, None, False, False] 0.1857730899070298


In [130]:
data = new_df.astype('float').values
	# data split
n_test = 3
# model configs
cfg_list = PredictorOptimize.exp_smoothing_configs(seasonal=[None])
# grid search

#walk_forward_validation(data, n_test, cfg)

scores = PredictorOptimize.grid_search(data[:,0], cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

> Model[['add', True, None, None, False, True]] 0.737
 > Model[['add', True, None, None, False, False]] 0.225
 > Model[['add', False, None, None, False, True]] 0.738
 > Model[['add', False, None, None, False, False]] 0.237
 > Model[[None, False, None, None, False, True]] 0.680
 > Model[[None, False, None, None, False, False]] 0.226
done
['add', True, None, None, False, False] 0.2249471841412778
[None, False, None, None, False, False] 0.22588994854105948
['add', False, None, None, False, False] 0.2370129759551018


In [145]:
sum(data)/len(data)

array([-4.69825])

In [150]:
PredictorOptimize.exp_smoothing_forecast(data, ['additive', True, 2, None, False, False])

-4.642129807658069

In [97]:
data

array([[-4.7593],
       [-4.7471],
       [-4.6321],
       [-4.6613],
       [-4.492 ],
       [-4.79  ],
       [-4.642 ],
       [-4.7312],
       [-4.8876],
       [-4.6399],
       [-4.439 ]])

In [98]:
new_df

,PARAMETER_ESTIMATE
Date,
2019-05-13,-4.7593
2019-06-13,-4.7471
2019-07-13,-4.6321
2019-08-13,-4.6613
2019-09-13,-4.492
2019-10-13,-4.79
2019-11-13,-4.642
2019-12-13,-4.7312
2020-01-13,-4.8876


In [99]:
import numpy as np
b=np.array([[PredictorOptimize.exp_smoothing_forecast(data, ['add', True, None, None, False, False])]])
data = np.concatenate((data, b))
#add([exp_smoothing_forecast(data, ['add', True, None, None, False, False])])
data

array([[-4.7593    ],
       [-4.7471    ],
       [-4.6321    ],
       [-4.6613    ],
       [-4.492     ],
       [-4.79      ],
       [-4.642     ],
       [-4.7312    ],
       [-4.8876    ],
       [-4.6399    ],
       [-4.439     ],
       [-4.44037225]])

In [100]:
PredictorOptimize.exp_smoothing_forecast(data, ['add', True, None, None, False, False])

-4.44037224892244

In [101]:
config = scores[:1][0][0]
config

"['add', True, None, None, False, False]"

In [127]:
n_test = 3
def predict_n_steps(data, n_steps, use_new_model = True):
    data_history = data
    cfg_list = PredictorOptimize.exp_smoothing_configs(seasonal=[0,6,12])
    scores = PredictorOptimize.grid_search(data[:,0], cfg_list, n_test)
    config = scores[:1][0][0]
    if use_new_model == True:
        for i in range(n_steps):
            new = np.array([[PredictorOptimize.exp_smoothing_forecast(data_history,ast.literal_eval(config))]])
            data_history = np.concatenate((data_history, new))
    else:
        t,d,s,p,b,r = ast.literal_eval(config)
        model = ExponentialSmoothing(data_history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
        model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
        for i in range(n_steps):            
            yhat = np.array([model_fit.predict(len(data_history), len(data_history))])            
            data_history = np.concatenate((data_history, yhat))            
    return data_history

In [128]:
predict_n_steps(data,4)

> Model[['add', True, None, 0, False, True]] 0.653
 > Model[['add', True, None, 0, False, False]] 0.205
 > Model[['add', True, None, 6, False, True]] 0.653
 > Model[['add', True, None, 6, False, False]] 0.205
 > Model[['add', True, None, 12, False, True]] 0.653
 > Model[['add', True, None, 12, False, False]] 0.205
 > Model[['add', False, None, 0, False, True]] 0.653
 > Model[['add', False, None, 0, False, False]] 0.214
 > Model[['add', False, None, 6, False, True]] 0.653
 > Model[['add', False, None, 6, False, False]] 0.214
 > Model[['add', False, None, 12, False, True]] 0.653
 > Model[['add', False, None, 12, False, False]] 0.214
 > Model[[None, False, 'add', 6, False, True]] 0.566
 > Model[[None, False, 'add', 6, False, False]] 0.659
 > Model[[None, False, None, 0, False, True]] 0.594
 > Model[[None, False, None, 0, False, False]] 0.206
 > Model[[None, False, None, 6, False, True]] 0.594
 > Model[[None, False, None, 6, False, False]] 0.206
 > Model[[None, False, None, 12, False, True

array([[-4.7593    ],
       [-4.7471    ],
       [-4.6321    ],
       [-4.6613    ],
       [-4.492     ],
       [-4.79      ],
       [-4.642     ],
       [-4.7312    ],
       [-4.8876    ],
       [-4.6399    ],
       [-4.439     ],
       [-4.44037225],
       [-4.44037225],
       [-2.9632595 ],
       [-3.0162364 ]])

In [129]:
predict_n_steps(data,4,False)

> Model[['add', True, None, 0, False, True]] 0.653
 > Model[['add', True, None, 0, False, False]] 0.205
 > Model[['add', True, None, 6, False, True]] 0.653
 > Model[['add', True, None, 6, False, False]] 0.205
 > Model[['add', True, None, 12, False, True]] 0.653
 > Model[['add', True, None, 12, False, False]] 0.205
 > Model[['add', False, None, 0, False, True]] 0.653
 > Model[['add', False, None, 0, False, False]] 0.214
 > Model[['add', False, None, 6, False, True]] 0.653
 > Model[['add', False, None, 6, False, False]] 0.214
 > Model[['add', False, None, 12, False, True]] 0.653
 > Model[['add', False, None, 12, False, False]] 0.214
 > Model[[None, False, 'add', 6, False, True]] 0.566
 > Model[[None, False, 'add', 6, False, False]] 0.659
 > Model[[None, False, None, 0, False, True]] 0.594
 > Model[[None, False, None, 0, False, False]] 0.206
 > Model[[None, False, None, 6, False, True]] 0.594
 > Model[[None, False, None, 6, False, False]] 0.206
 > Model[[None, False, None, 12, False, True

array([[-4.7593    ],
       [-4.7471    ],
       [-4.6321    ],
       [-4.6613    ],
       [-4.492     ],
       [-4.79      ],
       [-4.642     ],
       [-4.7312    ],
       [-4.8876    ],
       [-4.6399    ],
       [-4.439     ],
       [-4.44037225],
       [-4.44037225],
       [-4.44037225],
       [-4.44037225]])